In [1]:
import re
import nltk
import sklearn
import numpy as n
import pandas as pd
from nltk.stem import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#### Importing Datasets Primary and Secondary Dataset


In [2]:
#Primary Twitter.topic
df = pd.read_csv(r'primary_final\twitter_topic_lockdowns.csv')
df1 = pd.read_csv(r'primary_final\twitter_topic_masking_and_distancing.csv')
df2 = pd.read_csv(r'primary_final\twitter_topic_vaccination.csv')
#Secondary Change.org.topic
df_s = pd.read_csv(r'secondary_final\change.org_topic_lockdowns.csv')
df1_s = pd.read_csv(r'secondary_final\change.org_topic_masking_and_distancing.csv')
df2_s = pd.read_csv(r'secondary_final\change.org_topic_vaccination.csv')

In [3]:
df.count()

text           1200
Final Label    1200
dtype: int64

In [4]:
#Primary Lockdown
df.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,False
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,False
2,4 of the vaccines Jared bought are expected to...,False
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,False


In [5]:
df.describe()

,text,Final Label
count,1200,1200
unique,1195,2
top,STUDIES SHOW MORPHINE MILLIGRAM EQUIVALENT (MM...,False
freq,4,1145


In [6]:
#Primary Masking and Distancing
df1.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,False
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,False
2,4 of the vaccines Jared bought are expected to...,False
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [7]:
df1.describe()

,text,Final Label
count,1200,1200
unique,1195,2
top,STUDIES SHOW MORPHINE MILLIGRAM EQUIVALENT (MM...,False
freq,4,957


In [8]:
#Primary Vaccination
df2.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,True
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,True
2,4 of the vaccines Jared bought are expected to...,True
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [9]:
df2.describe()

,text,Final Label
count,1200,1200
unique,1195,2
top,STUDIES SHOW MORPHINE MILLIGRAM EQUIVALENT (MM...,True
freq,4,701


### Secondary Dataset

In [10]:
#Secondary Lockdown
df_s.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [11]:
df_s.describe()

,text,Final Label
count,1500,1500
unique,1500,2
top,Mthuli Ncube to remove 2% charge from all elec...,False
freq,1,1071


In [12]:
#Secondary Masking and Distancing
df1_s.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [13]:
df1_s.describe()

,text,Final Label
count,1500,1500
unique,1500,2
top,Mthuli Ncube to remove 2% charge from all elec...,False
freq,1,1429


In [14]:
#Secondary Vaccinatioon
df2_s.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [15]:
df2_s.describe()

,text,Final Label
count,1500,1500
unique,1500,2
top,Mthuli Ncube to remove 2% charge from all elec...,False
freq,1,1429


### Checking the count of Positive Classes

In [16]:
print("Primary Lockdowns:\n", df['Final Label'].value_counts(True)*100)
print("Primary Masking and Distancing:\n",df1['Final Label'].value_counts(True)*100)
print("Primary Vaccinations:\n",df2['Final Label'].value_counts(True)*100)
print("Secondary Lockdowns:\n",df_s['Final Label'].value_counts(True)*100)
print("Secondary Masking and Distancing:\n",df1_s['Final Label'].value_counts(True)*100)
print("Secondary Vaccinations:\n",df2_s['Final Label'].value_counts(True)*100)

Primary Lockdowns:
 False    95.416667
True      4.583333
Name: Final Label, dtype: float64
Primary Masking and Distancing:
 False    79.75
True     20.25
Name: Final Label, dtype: float64
Primary Vaccinations:
 True     58.416667
False    41.583333
Name: Final Label, dtype: float64
Secondary Lockdowns:
 False    71.4
True     28.6
Name: Final Label, dtype: float64
Secondary Masking and Distancing:
 False    95.266667
True      4.733333
Name: Final Label, dtype: float64
Secondary Vaccinations:
 False    95.266667
True      4.733333
Name: Final Label, dtype: float64


### Explanation to select right Primary dataset for training and modelling
#### From above we can see that in dataset twitter_topic_lockdowns and dataset twitter_topic_masking_and_distancing have low numbers of positive examples, as shown by frequency which represent number of False in datasets is 95% and 79% respectively. <br> 


### Therefore, we will select twitter_topic_vaccination.csv for Data partitioning and  Baseline modelling. Also, change.org_topic_vaccination from Secondary Dataset and designated 100% of SECONDARY dataset as the Test (SECONDARY) dataset. 




In [17]:
df_primary = df2.copy()
df_primary.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,True
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,True
2,4 of the vaccines Jared bought are expected to...,True
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [18]:
df_secondary = df2_s.copy()
df_secondary.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [19]:
df_secondary.isnull().sum()

text           0
Final Label    0
dtype: int64

#### Model 1 
#### Before Feature Engineering
1. Training dataset = Primary Data 70%
2. Test Dataset = Primary Data 30%


In [20]:
print(df_primary)

                                                   text  Final Label
0     Putin After Announcing #CovidVaccine #Russian ...         True
1     Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...         True
2     4 of the vaccines Jared bought are expected to...         True
3     One day you will realize CDC Guidelines magica...        False
4     Im far from lying.  Current CDC guidelines is ...         True
...                                                 ...          ...
1195  .Sprint To Develop A #COVIDVaccine – // https:...         True
1196  Here in the U.S. some localities have brought ...        False
1197  Sanitizer &amp; Mask Manufacturers After Russi...         True
1198  We are following all CDC guidelines through a ...        False
1199  People be losing hopes on covid vaccine like i...         True

[1200 rows x 2 columns]


In [21]:
X_prim = df_primary.drop(["Final Label"],axis = 1)
y_prim = df_primary["Final Label"]

X_sec = df_secondary.drop(["Final Label"],axis = 1)
y_sec = df_secondary["Final Label"]
print("Text: \n",X_prim)
print("\n")
print("Final Label:\n",y_prim)

Text: 
                                                    text
0     Putin After Announcing #CovidVaccine #Russian ...
1     Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...
2     4 of the vaccines Jared bought are expected to...
3     One day you will realize CDC Guidelines magica...
4     Im far from lying.  Current CDC guidelines is ...
...                                                 ...
1195  .Sprint To Develop A #COVIDVaccine – // https:...
1196  Here in the U.S. some localities have brought ...
1197  Sanitizer &amp; Mask Manufacturers After Russi...
1198  We are following all CDC guidelines through a ...
1199  People be losing hopes on covid vaccine like i...

[1200 rows x 1 columns]


Final Label:
 0        True
1        True
2        True
3       False
4        True
        ...  
1195     True
1196    False
1197     True
1198    False
1199     True
Name: Final Label, Length: 1200, dtype: bool


In [22]:
#split data to 30%test and 70% train
X_train_prim, X_test_prim, y_train_prim, y_test_prim = train_test_split(X_prim, y_prim, test_size = 0.3, random_state = 42)
print(X_train_prim.shape)
print(X_test_prim.shape)


(840, 1)
(360, 1)


In [23]:
training_data_f = pd.concat([X_train_prim,y_train_prim],axis=1)
training_data_f.head()

,text,Final Label
522,"I'm scared, too. We don't have remote availabl...",True
213,What kills me is I don't think they even belie...,False
120,UK vaccines receive a green signal after being...,True
917,"We would appreciate if the CDC, FDA, &amp; NIH...",True
1067,told me in an interview that the virus will bu...,False


In [24]:
test_data_f = pd.concat([X_test_prim,y_test_prim],axis=1)
test_data_f.head()

,text,Final Label
1178,Exciting news - barring any last minute safety...,True
865,Dr. Fauci speaks on CDC’s new guidelines for f...,True
101,"she says she is not anti-vax, just covid19 vax...",True
439,When will the mainstream media stop saying tha...,True
58,"If ANTIFA just means Anti-Fascism, than why di...",False


### Baseline Modeling: Train a simple bag-of-words classifier on Training dataset(training_data_f).

The bag of words model converts text into numerical representations (numerical feature vectors), which may then be used to train machine learning algorithms. The following are the main processes in putting together a bag-of-words model:
1. Using the whole collection of texts, create a vocabulary indices of words or tokens. The alphabetical arrangement of the vocabulary indices is a possibility.
2. Create a numerical feature vector for each document that indicates the frequency with which each word appears in various documents.

In [25]:
matrix = TfidfVectorizer(max_features=1000)
data_f_train = matrix.fit_transform(training_data_f['text'],training_data_f['Final Label']).toarray()
print(data_f_train.shape)


(840, 1000)


In [26]:
matrix = TfidfVectorizer(max_features=1000)
data_f_test = matrix.fit_transform(test_data_f['text'],test_data_f['Final Label']).toarray()
print(data_f_test.shape)

(360, 1000)


### Using Gaussian NB on Model 1

In [27]:
from sklearn.naive_bayes import GaussianNB
# classifier = RandomForestClassifier(n_estimators = 1000, random_state = 42)
classifier = GaussianNB()
classifier.fit(data_f_train, y_train_prim)

GaussianNB()

In [28]:
# predict class
y_pred_1 = classifier.predict(data_f_test)

# Confusion matrix
cm1 = confusion_matrix(y_test_prim, y_pred_1)
#Classification Report
cr1 = classification_report(y_test_prim, y_pred_1)
#Accuracy Score
accuracy1 = accuracy_score(y_test_prim, y_pred_1)
print("Confusion Matrix\n",cm1)
print("\nClassification Report\n", cr1)
print("\n Gaussian Accuracy for Primary Datasets (70% train and 30% test):", round(accuracy1*100,2))

Confusion Matrix
 [[ 60  79]
 [100 121]]

Classification Report
               precision    recall  f1-score   support

       False       0.38      0.43      0.40       139
        True       0.60      0.55      0.57       221

    accuracy                           0.50       360
   macro avg       0.49      0.49      0.49       360
weighted avg       0.52      0.50      0.51       360


 Gaussian Accuracy for Primary Datasets (70% train and 30% test): 50.28


#### Model 2
#### Before Feature Engineering
1. Training dataset = Primary Data 70%
2. Test Dataset = Secondary Data 100%


In [29]:
df_primary.head()

,text,Final Label
0,Putin After Announcing #CovidVaccine #Russian ...,True
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,True
2,4 of the vaccines Jared bought are expected to...,True
3,One day you will realize CDC Guidelines magica...,False
4,Im far from lying. Current CDC guidelines is ...,True


In [30]:
df_secondary.head()

,text,Final Label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [31]:
matrix1 = TfidfVectorizer(max_features=1000)
data_s_test = matrix1.fit_transform(df_secondary['text'],df_secondary['Final Label'])
print(data_s_test.shape)

(1500, 1000)


### Using GaussianNB on model 2

In [32]:
from sklearn.naive_bayes import GaussianNB
classifier1 = GaussianNB()
classifier1.fit(data_f_train, y_train_prim)

GaussianNB()

In [33]:
# predict class
y_pred_2 = classifier1.predict(data_s_test.toarray())
# Confusion matrix
cm2 = confusion_matrix(df_secondary['Final Label'], y_pred_2)
#Classification Report
cr2 = classification_report(df_secondary['Final Label'], y_pred_2)
#Accuracy Score
accuracy2 = accuracy_score(df_secondary['Final Label'], y_pred_2)
print("Confusion Matrix\n",cm2)
print("\nClassification Report\n", cr2)
print("\nGaussian Accuracy for Secondary (70% train and 100% test):", round(accuracy2*100,2))

Confusion Matrix
 [[718 711]
 [ 29  42]]

Classification Report
               precision    recall  f1-score   support

       False       0.96      0.50      0.66      1429
        True       0.06      0.59      0.10        71

    accuracy                           0.51      1500
   macro avg       0.51      0.55      0.38      1500
weighted avg       0.92      0.51      0.63      1500


Gaussian Accuracy for Secondary (70% train and 100% test): 50.67


From above it is clear that using Gaussian NB we will get 

1. Accuracy on Primary DataSet(Vaccination) Model 1 as ~50.3%

2. Accuracy on Secondary DataSet(Vaccination) Model 2 as ~50.7%


#### 4. Feature Engineering 

In [34]:
print('Text count Primary:',df_primary['text'].count())
print('\nText count Secondary:',df_secondary['text'].count())

Text count Primary: 1200

Text count Secondary: 1500


In [35]:
#Number of Characters
def count_chars(text):
    return len(text)

#Number of Words
def count_words(text):
    return len(text.split())

def count_sent(text):
    return len(nltk.sent_tokenize(text))

# #Number of Capital Characters
# def count_capital_chars(text):
#     count=0
#     for i in text:
#         if i.isupper():
#             count+=1
#     return count

def count_unique_words(text):
    return len(set(text.split()))

#Count of mentions
def count_mentions(text):
    x = re.findall(r'(@w[A-Za-z0-9]*)', text)
    return len(x)



In [36]:
def cleaning_pipe(sent):
    
    #Removing Stopwords and Punctuations
    data = re.sub('[^a-zA-Z]', ' ',sent)
    data = data.lower()
    data = data.split()
    data = [word for word in data if not word in set(stopwords.words('english'))]
    data = ' '.join(data)
    
    #Performing Stemming using PortarStemmer
    stemmer = PorterStemmer()
    words = word_tokenize(data)
    words = [ stemmer.stem(word) for word in words ]
    data = ' '.join(words)
    return data

In [37]:
df_primary['char_count'] = df_primary["text"].apply(lambda x:count_chars(x))
df_primary['word_count'] = df_primary["text"].apply(lambda x:count_words(x))
df_primary['sent_count'] = df_primary["text"].apply(lambda x:count_sent(x))
df_primary['avg_wordlength'] = df_primary['char_count']/df_primary['word_count']
df_primary['avg_sentlength'] = df_primary['word_count']/df_primary['sent_count']
df_primary['unique_word_count'] = df_primary["text"].apply(lambda x:count_unique_words(x))
df_primary['unique_vs_words'] = df_primary['unique_word_count']/df_primary['word_count']
df_primary["text"] = df_primary["text"].apply(cleaning_pipe)

In [38]:
df_primary.isnull().sum()

text                 0
Final Label          0
char_count           0
word_count           0
sent_count           0
avg_wordlength       0
avg_sentlength       0
unique_word_count    0
unique_vs_words      0
dtype: int64

In [39]:
df_primary

,text,Final Label,char_count,word_count,sent_count,avg_wordlength,avg_sentlength,unique_word_count,unique_vs_words
0,putin announc covidvaccin russian http co rndf...,True,69,6,1,11.500000,6.000000,6,1.000000
1,courtesi wa whatsapp covid covidvaccin http co...,True,68,6,2,11.333333,3.000000,6,1.000000
2,vaccin jare bought expect fail trump refus joi...,True,113,18,1,6.277778,18.000000,16,0.888889
3,one day realiz cdc guidelin magic align teache...,False,86,13,1,6.615385,13.000000,13,1.000000
4,im far lie current cdc guidelin wear mask indo...,True,113,20,3,5.650000,6.666667,20,1.000000
...,...,...,...,...,...,...,...,...,...
1195,sprint develop covidvaccin http co mp mqn,True,63,8,1,7.875000,8.000000,8,1.000000
1196,u local brought back mask mandat sever respect...,False,273,41,4,6.658537,10.250000,39,0.951220
1197,sanit amp mask manufactur russian vaccin annou...,True,135,12,1,11.250000,12.000000,12,1.000000
1198,follow cdc guidelin combin regular hand wash g...,False,173,24,1,7.208333,24.000000,22,0.916667


In [40]:
df_secondary['char_count'] = df_secondary["text"].apply(lambda x:count_chars(x))
df_secondary['word_count'] = df_secondary["text"].apply(lambda x:count_words(x))
df_secondary['sent_count'] = df_secondary["text"].apply(lambda x:count_sent(x))
df_secondary['avg_wordlength'] = df_secondary['char_count']/df_secondary['word_count']
df_secondary['avg_sentlength'] = df_secondary['word_count']/df_secondary['sent_count']
df_secondary['unique_word_count'] = df_secondary["text"].apply(lambda x:count_unique_words(x))
df_secondary['unique_vs_words'] = df_secondary['unique_word_count']/df_secondary['word_count']
df_secondary["text"] = df_secondary["text"].apply(cleaning_pipe)

In [41]:
df_secondary.isnull().sum()

text                 0
Final Label          0
char_count           0
word_count           0
sent_count           0
avg_wordlength       0
avg_sentlength       0
unique_word_count    0
unique_vs_words      0
dtype: int64

In [42]:
df_secondary

,text,Final Label,char_count,word_count,sent_count,avg_wordlength,avg_sentlength,unique_word_count,unique_vs_words
0,languag educ time covid,False,42,7,1,6.000000,7.0,7,1.000000
1,covid test kit,False,18,3,1,6.000000,3.0,3,1.000000
2,covid prison,False,18,4,1,4.500000,4.0,4,1.000000
3,get wale home,False,14,3,1,4.666667,3.0,3,1.000000
4,make pass fail avail mississippi state student...,False,88,12,1,7.333333,12.0,11,0.916667
...,...,...,...,...,...,...,...,...,...
1495,null void premier leagu,False,32,6,1,5.333333,6.0,6,1.000000
1496,free public transport nh staff accompani pay rise,False,60,11,1,5.454545,11.0,11,1.000000
1497,pm pleas give immedi covid insur cover nh staff,False,63,11,1,5.727273,11.0,11,1.000000
1498,say osha propos perman rule,False,38,7,1,5.428571,7.0,7,1.000000


In [43]:
X_pri = df_primary.drop(["Final Label"],axis = 1)
y_pri = df_primary["Final Label"]

X_sec = df_secondary.drop(["Final Label"],axis = 1)
y_sec = df_secondary["Final Label"]

### Model 1 after Feature Engineering

In [44]:
#Splitting the Dataset into Primary Train and Primary Test
X_pri_train,X_pri_test,y_pri_train,y_pri_test = train_test_split(X_pri,y_pri,test_size=0.3,random_state = 42)

In [45]:
X_pri_train.reset_index(inplace=True)
X_pri_test.reset_index(inplace=True)

X_pri_train = X_pri_train.drop("index",axis = 1)
X_pri_test = X_pri_test.drop("index",axis = 1)

### Creating the Dataframe for the TF-IDF Vectorizer

In [46]:
tf = TfidfVectorizer(max_features = 1000)

final_x_pri_train = tf.fit_transform(X_pri_train.text).toarray()
df = pd.DataFrame(final_x_pri_train)  
final_x_pri_train = X_pri_train.join(df) 
print("The size of Primary train with feature Engineering is:",final_x_pri_train.shape)


final_x_pri_test = tf.transform(X_pri_test.text).toarray()
df1 = pd.DataFrame(final_x_pri_test)  
final_x_pri_test = X_pri_test.join(df1) 
print("The size of Primary test with feature Engineering is:",final_x_pri_test.shape)


final_x_sec_test = tf.transform(X_sec.text).toarray()
df2 = pd.DataFrame(final_x_sec_test)  
final_x_sec_test = X_sec.join(df2) 
print("The size of Primary test with feature Engineering is:",final_x_sec_test.shape)

The size of Primary train with feature Engineering is: (840, 1008)
The size of Primary test with feature Engineering is: (360, 1008)
The size of Primary test with feature Engineering is: (1500, 1008)


In [47]:
final_x_pri_train = final_x_pri_train.drop("text",axis = 1)
final_x_pri_test = final_x_pri_test.drop("text",axis = 1)
final_x_sec_test = final_x_sec_test.drop("text",axis = 1)


In [48]:
clf = GaussianNB()
clf.fit(final_x_pri_train, y_pri_train)
y_pred_pri = clf.predict(final_x_pri_test)

print("The Classification Report :\n",classification_report(y_pri_test,y_pred_pri))
print("\nConfusion Matrix : \n",confusion_matrix(y_pri_test,y_pred_pri))
print("\nThe Accuracy in Primary Set is:",round((accuracy_score(y_pri_test,y_pred_pri))*100,2))

The Classification Report :
               precision    recall  f1-score   support

       False       0.63      0.65      0.64       139
        True       0.78      0.76      0.77       221

    accuracy                           0.72       360
   macro avg       0.70      0.71      0.70       360
weighted avg       0.72      0.72      0.72       360


Confusion Matrix : 
 [[ 91  48]
 [ 54 167]]

The Accuracy in Primary Set is: 71.67


### Model 2 after Feature Engineering

### Model using GaussianNB

In [49]:
clf = GaussianNB()
clf.fit(final_x_pri_train, y_pri_train)
y_pred_sec = clf.predict(final_x_sec_test)

print("The Classification Report: \n",classification_report(y_sec,y_pred_sec))
print("\nConfusion Matrix : \n",confusion_matrix(y_sec,y_pred_sec))
print("\nThe Accuracy in Secondary Set using Gaussian NB is:",round((accuracy_score(y_sec,y_pred_sec))*100,2))

The Classification Report: 
               precision    recall  f1-score   support

       False       0.94      0.56      0.70      1429
        True       0.03      0.30      0.06        71

    accuracy                           0.55      1500
   macro avg       0.49      0.43      0.38      1500
weighted avg       0.90      0.55      0.67      1500


Confusion Matrix : 
 [[799 630]
 [ 50  21]]

The Accuracy in Secondary Set using Gaussian NB is: 54.67


From above it is clear that using Gaussian NB we will get 

1. Accuracy on Primary DataSet(Vaccination) Model 1 as ~71.67%

2. Accuracy on Secondary DataSet(Vaccination) Model 2 as ~54.7%

